### Libraries

In [1]:
# Importing os.
import os

# Importing urllib.request.
import urllib.request

# Importing zipfile.
import zipfile

# Importing pandas.
import pandas as pd

# Importing numpy.
import numpy as np

# Importing random.
import random

# Importing tensorflow.
import tensorflow as tf

# Importing train_test_split.
from sklearn.model_selection import train_test_split

# Importing backend.
from keras import backend

# Importing set_random_seed.
from keras.utils import set_random_seed

# Importing pad_sequences.
from keras_preprocessing.sequence import pad_sequences

# Importing Sequential.
from keras.models import Sequential

# Importing Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation.
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation

# Importing Adam.
from keras.optimizers import Adam

# Importing classification_report.
from sklearn.metrics import classification_report

# Sets reproducibility.
def set_reproducibility(seed):

  # Setting seeds.
  random.seed(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)
  os.environ["TF_DETERMINISTIC_OPS"] = "1"

# Setting seed.
set_reproducibility(seed = 42)

### Data Preprocessing

In [2]:
# Function used to download .zips.
def downloader(url, folder_name, filename):

  # Defining data folder path.
  data_path = os.path.join(os.getcwd(), folder_name)

  # Creating data folder.
  if not os.path.exists(data_path):
      os.makedirs(data_path)

  # Defining .zip file path.
  zip_path = os.path.join(os.getcwd(), folder_name, filename)

  # Requesting .zip file.
  if not os.path.exists(zip_path):
      urllib.request.urlretrieve(url, zip_path)

  # Extracting data from .zip.
  with zipfile.ZipFile(zip_path, "r") as zip_ref:
      zip_ref.extractall(path = data_path)

  # Returning data_path and zip_path.
  return data_path, zip_path

In [3]:
# Downloading dataset.
data_path, _ = downloader(url = "https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip", folder_name = "data", filename = "dependency_treebank.zip")

# Downloading glove.
glove_path, _ = downloader(url = "https://nlp.stanford.edu/data/glove.6B.zip", folder_name = "glove", filename = "glove.6B.zip")

In [4]:
# Defining the dataset name.
dataset_name = "dependency_treebank"

# Defining path to first training sample.
file_path = os.path.join(data_path, dataset_name, "wsj_0001.dp")

# Reading first training sample.
if os.path.isfile(file_path):

  # Printing file.
  with open(file_path, mode = "r") as text_file: print(text_file.read())

Pierre	NNP	2
Vinken	NNP	8
,	,	2
61	CD	5
years	NNS	6
old	JJ	2
,	,	2
will	MD	0
join	VB	8
the	DT	11
board	NN	9
as	IN	9
a	DT	15
nonexecutive	JJ	15
director	NN	12
Nov.	NNP	9
29	CD	16
.	.	8

Mr.	NNP	2
Vinken	NNP	3
is	VBZ	0
chairman	NN	3
of	IN	4
Elsevier	NNP	7
N.V.	NNP	12
,	,	12
the	DT	12
Dutch	NNP	12
publishing	VBG	12
group	NN	5
.	.	3



In [5]:
# Defining embedding size.
EMBEDDING_SIZE = 50

# Defining specific glove's file path.
glove_file = os.path.join(os.getcwd(), glove_path, f"glove.6B.{str(EMBEDDING_SIZE)}d.txt")

# Reading lines of file.
with open(glove_file, encoding = "utf8" ) as text_file: 
  lines = text_file.readlines()

# Defining initial vocabulary.
embedding_vocabulary = {}

# Reading single lines.
for line in lines:

  # Splitting line.
  splits = line.split()

  # Storing line into vocabulary.
  embedding_vocabulary[splits[0]] = np.array([float(val) for val in splits[1:]])

# Printing one entry of the vocabulary.
print("The embedding for 'the' is:\n{}.".format(embedding_vocabulary["the"]))

The embedding for 'the' is:
[ 4.1800e-01  2.4968e-01 -4.1242e-01  1.2170e-01  3.4527e-01 -4.4457e-02
 -4.9688e-01 -1.7862e-01 -6.6023e-04 -6.5660e-01  2.7843e-01 -1.4767e-01
 -5.5677e-01  1.4658e-01 -9.5095e-03  1.1658e-02  1.0204e-01 -1.2792e-01
 -8.4430e-01 -1.2181e-01 -1.6801e-02 -3.3279e-01 -1.5520e-01 -2.3131e-01
 -1.9181e-01 -1.8823e+00 -7.6746e-01  9.9051e-02 -4.2125e-01 -1.9526e-01
  4.0071e+00 -1.8594e-01 -5.2287e-01 -3.1681e-01  5.9213e-04  7.4449e-03
  1.7778e-01 -1.5897e-01  1.2041e-02 -5.4223e-02 -2.9871e-01 -1.5749e-01
 -3.4758e-01 -4.5637e-02 -4.4251e-01  1.8785e-01  2.7849e-03 -1.8411e-01
 -1.1514e-01 -7.8581e-01].


In [6]:
# List containing dataframe rows.
dataframe_rows = []

# List containing words of a single sentence.
row_words = []

# List containing tags of a single sentence.
row_tags = []

# Defining data folder path.
folder = os.path.join(data_path, dataset_name)

# Storing rows.
for filename in sorted(os.listdir(folder)):

  # Computing path to file.
  file_path = os.path.join(folder, filename)

  # Checking existance of file.
  if os.path.isfile(file_path):

    # Opening the file.
    with open(file_path, mode = "r") as text_file:

      # Reading lines.
      while True:

        # Reading next line.
        line = text_file.readline()

        # Checking that line is different from "\n" (empty line) and from last line (EOF).
        if line and line != "\n":

          # Storing the word.
          row_words.append(line.split()[0])

          # Storing the POS tag.
          row_tags.append(line.split()[1])

        else:

          # Creating a row.
          dataframe_row = {"file_id": int(filename.split(".")[0].split("_")[1]), "sentence": row_words, "tags": row_tags}

          # Appending row.
          dataframe_rows.append(dataframe_row)

          # Resetting row_words list so to store a new sentence.
          row_words = []

          # Resetting row_tags list so to store a new sentence.
          row_tags = []

          # If, in particular, EOF is reached, then break the inner loop.
          if not line: break

# Creating pandas dataframe.
dataframe = pd.DataFrame(dataframe_rows)

# Printing dataframe head.
dataframe.head()

,file_id,sentence,tags
0,1,"[Pierre, Vinken, ,, 61, years, old, ,, will, j...","[NNP, NNP, ,, CD, NNS, JJ, ,, MD, VB, DT, NN, ..."
1,1,"[Mr., Vinken, is, chairman, of, Elsevier, N.V....","[NNP, NNP, VBZ, NN, IN, NNP, NNP, ,, DT, NNP, ..."
2,2,"[Rudolph, Agnew, ,, 55, years, old, and, forme...","[NNP, NNP, ,, CD, NNS, JJ, CC, JJ, NN, IN, NNP..."
3,3,"[A, form, of, asbestos, once, used, to, make, ...","[DT, NN, IN, NN, RB, VBN, TO, VB, NNP, NN, NNS..."
4,3,"[The, asbestos, fiber, ,, crocidolite, ,, is, ...","[DT, NN, NN, ,, NN, ,, VBZ, RB, JJ, IN, PRP, V..."


In [7]:
# Defining training split.
TRAINING_SPLIT = range(1, 101)

# Defining validation split.
VALIDATION_SPLIT = range(101, 151)

# Defining test split.
TEST_SPLIT = range(151, 200)

# Computing train dataframe.
train = dataframe.loc[dataframe["file_id"].isin(TRAINING_SPLIT)]

# Computing validation dataframe.
validation = dataframe.loc[dataframe["file_id"].isin(VALIDATION_SPLIT)]

# Computing test dataframe.
test = dataframe.loc[dataframe["file_id"].isin(TEST_SPLIT)]

In [8]:
# Overall words.
words = []

# Overall tags.
tags = []

# Extending words with train words.
words.extend([item.lower() for sublist in train["sentence"].tolist() for item in sublist])

# Extending words with validation words.
words.extend([item.lower() for sublist in validation["sentence"].tolist() for item in sublist])

# Extending words with test words.
words.extend([item.lower() for sublist in test["sentence"].tolist() for item in sublist])

# Extending tags with train tags.
tags.extend([item for sublist in train["tags"].tolist() for item in sublist])

# Extending tags with validation tags.
tags.extend([item for sublist in validation["tags"].tolist() for item in sublist])

# Extending tags with test tags.
tags.extend([item for sublist in test["tags"].tolist() for item in sublist])

# Removing duplicates from words. By using a dict instead of a set I can get reproducible results.
words = list(dict.fromkeys(words))

# Removing duplicates from tags. By using a dict instead of a set I can get reproducible results.
tags = list(dict.fromkeys(tags))

# Building vocabulary for words.
word_to_index = {}
word_to_index["PAD"] = 0
word_to_index["OOV"] = 1
for i, word in enumerate(list(words)): word_to_index[word] = i + 2

# Building vocabulary for tags.
tag_to_index = {}
tag_to_index["PAD"] = 0
for i, tag in enumerate(list(tags)): tag_to_index[tag] = i + 1

In [9]:
word_to_index["the"]

10

In [10]:
# Function used to transform sentences into vectors of integers using the word_to_index vocabulary.
def convert_sentences(input_sentences, vocabulary):

  # Output sentences.
  output_sentences = []

  # Converting input sentences.
  for sentence in input_sentences:

    # List to store the words' indexes.
    sentence_indexes = []

    # Corventing each word in sentence.
    for word in sentence:

      try:

        # Computing index.
        sentence_indexes.append(vocabulary[word.lower()])

      except KeyError:

        # Computing index.
        sentence_indexes.append(vocabulary["OOV"])

    # Appending list of indexes to training set.
    output_sentences.append(sentence_indexes)

  # Returning output_sentences.
  return output_sentences

# Function used to transform list of tags into vectors of integers using the tag_to_index vocabulary.
def convert_tags(input_tags, vocabulary):

  # Output tags.
  output_tags = []

  # Converting input tags.
  for tags_list in input_tags:

    # Computing index.
    output_tags.append([vocabulary[tag] for tag in tags_list])

  # Returning output_tags.
  return output_tags

In [11]:
# Computing train sentences.
train_sentences = convert_sentences(input_sentences = train["sentence"].tolist(), vocabulary = word_to_index)

# Computing validation sentences.
validation_sentences = convert_sentences(input_sentences = validation["sentence"].tolist(), vocabulary = word_to_index)

# Computing test sentences.
test_sentences = convert_sentences(input_sentences = test["sentence"].tolist(), vocabulary = word_to_index)

# Computing train tags.
train_tags = convert_tags(input_tags = train["tags"].tolist(), vocabulary = tag_to_index)

# Computing validation tags.
validation_tags = convert_tags(input_tags = validation["tags"].tolist(), vocabulary = tag_to_index)

# Computing test tags.
test_tags = convert_tags(input_tags = test["tags"].tolist(), vocabulary = tag_to_index)

In [12]:
print(train_sentences[0])
print(train_tags[0])

[2, 3, 4, 5, 6, 7, 4, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
[1, 1, 2, 3, 4, 5, 2, 6, 7, 8, 9, 10, 8, 5, 9, 1, 3, 11]


In [47]:
# Computing length of longest train sentence.
MAX_LENGTH = len(max(max(train["sentence"].tolist(), key = len), 
                     max(validation["sentence"].tolist(), key = len), 
                     max(test["sentence"].tolist(), key = len)))

# Padding train sentences.
train_sentences = pad_sequences(train_sentences, maxlen = MAX_LENGTH, padding = "post")

# Padding validation sentences.
validation_sentences = pad_sequences(validation_sentences, maxlen = MAX_LENGTH, padding = "post")

# Padding test sentences.
test_sentences = pad_sequences(test_sentences, maxlen = MAX_LENGTH, padding = "post")

# Padding train tags.
train_tags = pad_sequences(train_tags, maxlen = MAX_LENGTH, padding = "post")

# Padding validation tags.
validation_tags = pad_sequences(validation_tags, maxlen = MAX_LENGTH, padding = "post")

# Padding test tags.
test_tags = pad_sequences(test_tags, maxlen = MAX_LENGTH, padding = "post")

In [14]:
# Number of converted words.
hits = 0

# Number of OOV words.
misses = 0

# Preparing embedding matrix so that OOV words will be assigned a vector of zeros.
embedding_matrix = np.zeros((len(word_to_index), int(EMBEDDING_SIZE)))

# Retrieving embedding vector for each word.
for word, i in word_to_index.items():

  # Computing vector.
  vector = embedding_vocabulary.get(word)

  # Checking vector.
  if vector is not None:
    
    # Populating the embedding matrix.
    embedding_matrix[i] = vector

    # Increasing number of hits.
    hits += 1

  else:

    # Increasing number of misses.
    misses += 1

# Printing number of hits and missed.
print(f"Converted {hits} words. Missed {misses} words.")

Converted 10271 words. Missed 678 words.


In [15]:
embedding_matrix[word_to_index["the"]]

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01])

In [16]:
# Function used to one hot encode tag lists.
def one_hot_encoder(tags_lists, n_categories):

  # List of encodings.
  one_hot_encodings = []

  # Iterating over lists of tags (e.g. train_tags).
  for tags_list in tags_lists:

    # One-hot encoding of the specific list.
    one_hot_encoding = []

    # Iterating over tags
    for tag in tags_list:

      # Creating vector of zeros.
      one_hot_encoding.append(np.zeros(n_categories))

      # Storing a one in the tag position.
      one_hot_encoding[-1][tag] = 1.0

    # Appending the one hot encoding.
    one_hot_encodings.append(one_hot_encoding)
  
  # Returning the list of encodings.
  return np.array(one_hot_encodings)

In [17]:
# One-hot encoding train tags.
train_tags = one_hot_encoder(tags_lists = train_tags, n_categories = len(tag_to_index))

# One-hot encoding validation tags.
validation_tags = one_hot_encoder(tags_lists = validation_tags, n_categories = len(tag_to_index))

# One-hot encoding test tags.
test_tags = one_hot_encoder(tags_lists = test_tags, n_categories = len(tag_to_index))

### Model Definition and Training

In [18]:
# Sequential model.
model = Sequential()

# Input layer.
model.add(InputLayer(input_shape = (MAX_LENGTH,)))

# Embedding layer defined by embedding_matrix.
model.add(Embedding(len(word_to_index), EMBEDDING_SIZE, weights = [embedding_matrix], trainable = False))

# Bi-LSTM.
model.add(Bidirectional(LSTM(256, return_sequences = True)))

# Dense layer.
model.add(TimeDistributed(Dense(len(tag_to_index))))

# Softmax.
model.add(Activation("softmax"))

# Compiling.
model.compile(loss = "categorical_crossentropy", optimizer = Adam(), metrics = ["accuracy"])

# Printing summary.
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 249, 50)           547450    
                                                                 
 bidirectional (Bidirectiona  (None, 249, 512)         628736    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 249, 46)          23598     
 ibuted)                                                         
                                                                 
 activation (Activation)     (None, 249, 46)           0         
                                                                 
Total params: 1,199,784
Trainable params: 652,334
Non-trainable params: 547,450
_________________________________________________________________


In [19]:
# Fitting the model.
model.fit(train_sentences, train_tags, batch_size = 256, epochs = 40, validation_data = (validation_sentences, validation_tags))

Epoch 1/40
8/8 [==============================] - 7s 281ms/step - loss: 2.8638 - accuracy: 0.8613 - val_loss: 0.5109 - val_accuracy: 0.9072
Epoch 2/40
8/8 [==============================] - 1s 144ms/step - loss: 0.4114 - accuracy: 0.9113 - val_loss: 0.3403 - val_accuracy: 0.9146
Epoch 3/40
8/8 [==============================] - 1s 145ms/step - loss: 0.3217 - accuracy: 0.9175 - val_loss: 0.2973 - val_accuracy: 0.9223
Epoch 4/40
8/8 [==============================] - 1s 145ms/step - loss: 0.2869 - accuracy: 0.9234 - val_loss: 0.2746 - val_accuracy: 0.9241
Epoch 5/40
8/8 [==============================] - 1s 145ms/step - loss: 0.2679 - accuracy: 0.9276 - val_loss: 0.2601 - val_accuracy: 0.9321
Epoch 6/40
8/8 [==============================] - 1s 147ms/step - loss: 0.2548 - accuracy: 0.9325 - val_loss: 0.2475 - val_accuracy: 0.9350
Epoch 7/40
8/8 [==============================] - 1s 146ms/step - loss: 0.2424 - accuracy: 0.9373 - val_loss: 0.2355 - val_accuracy: 0.9407
Epoch 8/40
8/8 [====

In [49]:
# Computing predictions.
pred = model.predict(validation_sentences)

# Computing classification report.
report = classification_report(np.argmax(validation_tags, axis = 2).flatten(), 
                               np.argmax(pred, axis = 2).flatten(), 
                               labels = np.arange(0, 46, 1),
                               target_names = list(tag_to_index.keys()),
                               zero_division = 0,
                               output_dict = True)

# Macro F1-score without punctuation classes.
macro_f1 = 0

# Punctuation tag list.
punctuation_tag_list = ["PAD", ",", ".", "``", "''", ":", "$", "#"]

# Iterating over classes.
for tag in list(tag_to_index.keys()):

  # Updating the macro F1-score.
  if tag not in punctuation_tag_list: macro_f1 = macro_f1 + report[tag]["f1-score"]

# Dividing macro F1-score with the number of non-punctuation classes.
macro_f1 = macro_f1 / (len(list(tag_to_index.keys())) - len(punctuation_tag_list))

# Macro F1-score.
print(f"Macro F1-score with no punctuation classes: {macro_f1}.")

41/41 [==============================] - 1s 16ms/step
Macro F1-score with no punctuation classes: 0.4452328534748629.
